In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from torch.utils.data import DataLoader

input=load_dataset('glue','stsb')

checkpoint='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=1)

def token_data(lines):
  return tokenizer(lines['sentence1'],lines['sentence2'],truncation=True)

data_collator=DataCollatorWithPadding(tokenizer)

tokenized_data=input.map(token_data, batched=True)
tokenized_data=tokenized_data.remove_columns(['sentence1','sentence2','idx'])
tokenized_data=tokenized_data.rename_column('label','labels')
tokenized_data.set_format('torch')
tokenized_data['train'].column_names

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
train_data=DataLoader(tokenized_data['train'], shuffle=True, batch_size=10, collate_fn=data_collator)
eval_data=DataLoader(tokenized_data['validation'], batch_size=10, collate_fn=data_collator)

for batch in train_data:
  break
{k: v.shape for k,v in batch.items()}

{'labels': torch.Size([10]),
 'input_ids': torch.Size([10, 33]),
 'token_type_ids': torch.Size([10, 33]),
 'attention_mask': torch.Size([10, 33])}

In [ ]:
model_out=model(**batch)
print(model_out.loss, model_out.logits.shape)

tensor(11.0031, grad_fn=<MseLossBackward0>) torch.Size([10, 1])


In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs=3
num_training_steps = num_epochs*(len(train_data))
lr_scheduler=get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
print(num_training_steps)

1725


In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm
progress_bar=tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_data:
    batch={k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1725 [00:00<?, ?it/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
from evaluate import load

metric=load('glue','stsb')
model.eval()
for batch in eval_data:
  batch={k: v.to(device) for k,v in batch.items()}
  with torch.no_grad():
    output = model(**batch)
  logits=output.logits
  metric.add_batch(predictions=logits, references=batch['labels'])
metric.compute()

{'pearson': 0.8888921529118027, 'spearmanr': 0.8853017792084178}

# Using Accelerate:
Using the 🤗 Accelerate library, with just a few adjustments we can enable distributed training on multiple GPUs or TPUs. Starting from the creation of the training and validation dataloaders

In [ ]:
#import torch
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#model.to(device)
#device

from accelerate import Accelerator
accelerator = Accelerator()
train_data, eval_data, model, optimizer=accelerator.prepare(train_data, eval_data, model, optimizer)




In [ ]:
from tqdm.auto import tqdm
progress_bar=tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_data:
    #batch={k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss

    #loss.backward()
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1725 [00:00<?, ?it/s]